The important idea for the gate keeper to work is to have custom rules for suricata to monitor what and how much traffic is moving throught the net work. I can't modify fire walls with suricata since it is and Intrusion Detction System but I can put in rules for suricata so it can read lots of instances in the logs and then use an active "firewall" pyhton script to ban ips that are trying to flood the system. 

Suricata rules that are added to /etc/suricata/rules/custom.rules :

# SYN Flood Detection Rule
alert tcp any any -> any any (msg:"SYN Flood Detected"; flow:stateless; flags:S; threshold: type both, track by_src, count 100, seconds 1; sid:1000001; rev:1;)

# UDP Flood Detection Rule
alert udp any any -> any any (msg:"UDP Flood Detected"; flow:stateless; threshold: type both, track by_src, count 1000, seconds 1; sid:1000002; rev:1;)

This is the start of the Gate Keeper Extra credit. The goal of this assignment is to use suricata to monitor traffic and to rate limit the number of syn connections to my own computer with my own ip address which is 192.168.0.184/24, but when running the code on your own please change to your own ip addr after downloading and enabling suricata.

To make sure that suricata correctly loads follow instructions from Project2_part2.2.ipynb and use this essential command to read from the right interface and to load the correct config file
sudo suricata -i virbr0 -c //etc/suricata/suricata.yaml

In [2]:
import json
import subprocess
import time
from collections import defaultdict
import threading

The strategy that we will be using is that we will be reading the suricata log to then decide what actions to take

In [3]:
# Path to Suricata's eve.json log file
EVE_LOG_PATH = "/var/log/suricata/eve.json"

# Thresholds for blocking (adjust based on your needs)
SYN_FLOOD_THRESHOLD = 100  # Number of SYN packets per second
UDP_FLOOD_THRESHOLD = 1000  # Number of UDP packets per second
TIME_WINDOW = 1  # Time window in seconds

# Track IPs and their packet counts
ip_syn_count = defaultdict(int)
ip_udp_count = defaultdict(int)

using ip tabels we will block ips  

In [4]:
def block_ip(ip):
    """Block an IP address using iptables."""
    try:
        subprocess.run(["iptables", "-A", "INPUT", "-s", ip, "-j", "DROP"], check=True)
        print(f"Blocked IP: {ip}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to block IP {ip}: {e}")

now that we have code to block and unblock ips we now need to monitor the logs to see who should be blocked and when 

In [5]:
def monitor_log():
    """Monitor the Suricata eve.json log file for flood alerts."""
    with open(EVE_LOG_PATH, "r") as logfile:
        logfile.seek(0, 2)  # Move to the end of the file
        while True:
            line = logfile.readline()
            if line:
                try:
                    event = json.loads(line)
                    if event["event_type"] == "alert":
                        alert = event["alert"]
                        src_ip = event["src_ip"]
                        if "SYN Flood Detected" in alert["signature"]:
                            ip_syn_count[src_ip] += 1
                        elif "UDP Flood Detected" in alert["signature"]:
                            ip_udp_count[src_ip] += 1
                except json.JSONDecodeError:
                    continue
            else:
                time.sleep(0.1)  # Sleep briefly to avoid high CPU usage

def check_thresholds():
    """Check if any IPs have exceeded the flood thresholds."""
    while True:
        for ip, count in ip_syn_count.items():
            if count >= SYN_FLOOD_THRESHOLD:
                block_ip(ip)
                ip_syn_count[ip] = 0  # Reset the count after blocking
        for ip, count in ip_udp_count.items():
            if count >= UDP_FLOOD_THRESHOLD:
                block_ip(ip)
                ip_udp_count[ip] = 0  # Reset the count after blocking
        time.sleep(TIME_WINDOW)

now we just have to run the code to make it active

In [6]:
def main():
    """Main function to start the script."""
    # Start the log monitoring thread
    log_thread = threading.Thread(target=monitor_log)
    log_thread.daemon = True
    log_thread.start()

    # Start the threshold checking thread
    threshold_thread = threading.Thread(target=check_thresholds)
    threshold_thread.daemon = True
    threshold_thread.start()

    # Keep the main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Script terminated by user.")

In [ ]:
# Call the main function to start the script
main()

Blocked IP: 192.168.124.2
